In [1]:
import pandas as pd 
import numpy as np

from pandas.testing import assert_series_equal, assert_frame_equal

import os
import re

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
def reader(file):
    try:
        return pd.read_csv(file)
    except UnicodeDecodeError:
        return pd.read_csv(file, encoding='iso-8859-1')

In [293]:
os.listdir('new')

['my_billing.csv',
 'cleaned_usage_df.csv',
 'cleaned_billing_df.csv',
 '.DS_Store',
 'cleaned_pai_snapshot_c3_d2018-08-08_support.csv',
 'cleaned_pai_snapshot_c3_d2018-08-08_account.csv',
 'cleaned_account_df.csv',
 'cleaned_pai_snapshot_c3_d2018-08-08_target.csv',
 'cleaned_pai_snapshot_c3_d2018-08-08_contact.csv',
 'my_target.csv',
 'cleaned_engagement_df.csv',
 'my_engagement.csv',
 'cleaned_pai_snapshot_c3_d2018-08-08_engagement.csv',
 'my_contact.csv',
 'cleaned_target_df.csv',
 'my_account.csv',
 'cleaned_contact_df.csv',
 'my_usage.csv',
 'cleaned_support_df.csv',
 'c3',
 'cleaned_pai_snapshot_c3_d2018-08-08_billing.csv',
 'cleaned_pai_snapshot_c3_d2018-08-08_usage.csv',
 'my_support.csv']

In [294]:
os.listdir()

['.DS_Store',
 'old',
 'scratchpad.ipynb',
 'trying to do it myself.ipynb',
 'new',
 '.ipynb_checkpoints']

In [292]:
os.listdir('old')

['raw_cases_CLICKTALE.csv',
 'raw_support_CLICKTALE.csv',
 '.DS_Store',
 'cleaned_support_CLICKTALE.csv',
 'cleaned_engagement_CLICKTALE.csv',
 'raw_contract_details_CLICKTALE.csv',
 'raw_clicktale_accounts_CLICKTALE.csv',
 'raw_billing_CLICKTALE.csv',
 'cleaned_account_CLICKTALE.csv',
 'cleaned_usage_CLICKTALE.csv',
 'raw_usage_file_CLICKTALE.csv',
 'raw_core_monthly_recordings_CLICKTALE.csv',
 'raw_accounts_details_CLICKTALE.csv',
 'cleaned_target_CLICKTALE.csv',
 'cleaned_billing_CLICKTALE.csv']

In [69]:
dt_cols = [
    'created_date',
    'SampleDate',
    'created_date_timestamp',
    'date', 
    'recipe_execution_date', 
    'churn_probability_date',
    'next_calculated_exit_point',
    'Churned Date', 
    'poc_end_date',
    'account_birth_date'
]

In [291]:
reader('target_clean.csv')

FileNotFoundError: File b'target_clean.csv' does not exist

In [127]:
table = 'engagement'

In [128]:
df = reader('new/cleaned_pai_snapshot_c3_d2018-08-08_' + table + '.csv')
# df.drop(columns=['Unnamed: 0'], inplace=True) # index_col
orig = reader('new/cleaned_' + table + '_df.csv')

In [129]:
df.equals(orig)

False

In [130]:
for col in dt_cols:
    if col in list(df):
        print(col)
        try:
            assert_series_equal(df[col], orig[col])
            continue
        except AssertionError:
            try:
                assert_series_equal(df[col], orig[col].map(lambda x: x[:-9] if type(x) == str else x))
                orig[col] = orig[col].map(lambda x: x[:-9] if type(x) == str else x)
                continue
            except AssertionError:
                try:
                    assert_series_equal(df[col].map(lambda x: x[:-9] if type(x) == str else x), orig[col])
                    df[col] = df[col].map(lambda x: x[:-9] if type(x) == str else x)
                    continue
                except AssertionError:
                    raise

SampleDate
recipe_execution_date


In [131]:
df.equals(orig)

True

Next step, if this fails - go through df series by series

In [132]:
for i in list(zip(*(list(df), list(orig)))):
    assert i[0] == i[1]
    col = i[0]
    print(col)
    assert_series_equal(df[col], orig[col])

Login
SampleDate
TotangoModule
TotangoModuleCount
pai_account_id
client_id
recipe_execution_date


In [133]:
assert_frame_equal(df, orig)

Especially for target, which has a change in is_deleted

In [122]:
print(orig.is_deleted.unique(), df.is_deleted.unique())

[nan] [0]


In [280]:
list(s)

[1.0, 3.0, nan, nan, 4.0, 6.0, nan, nan, nan, nan, 2.0]

In [281]:
df = pd.DataFrame({'AAA': [4, 5, 6, 7],'BBB': [10, 20, 30, 40],'CCC': [100, 50, -30, -50]})

In [282]:
df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [283]:
aValue = 43.0

In [289]:
type((df.CCC - aValue).abs().argsort())

pandas.core.series.Series

In [290]:
df.loc[pd.Series([1,0,2,3])]

,AAA,BBB,CCC
1,5,20,50
0,4,10,100
2,6,30,-30
3,7,40,-50


Finding streaks of NaN

In [134]:
import numpy as np

In [279]:
s = pd.Series([1,3,np.nan, np.nan, 4,6,np.nan, np.nan, np.nan, np.nan, 2])
arr = np.array(s)

In [189]:
arr = np.nan_to_num(arr)

In [190]:
df = pd.DataFrame(arr)

In [195]:
df2 = pd.DataFrame(arr)

In [277]:
s = pd.Series([1,3,np.nan, np.nan, 4,6,np.nan, np.nan, np.nan, np.nan, 2])
s = pd.Series([np.nan, np.nan, 4,np.nan, np.nan, 7, np.nan, np.nan, np.nan, np.nan, 1, np.nan, np.nan, 6, np.nan, np.nan])
df = pd.DataFrame(s)
df2 = df[df.isna().any(axis=1)]
grouped = df2.groupby((df2.index.to_series().diff() > 1).cumsum())
df3 = pd.DataFrame()
for group in grouped:
    test = group[1].reset_index()
    test['fill'] = pd.qcut(test.iloc[:,0], 2, labels=['bf', 'ff'])
    df3 = pd.concat([df3, test])
df3.set_index('index', inplace=True)
pd.merge(df, df3, how='left', left_index=True, right_index=True).drop(columns=['0_y'])

,0_x,fill
0,NaN,bf
1,NaN,ff
2,4.0,NaN
3,NaN,bf
4,NaN,ff
5,7.0,NaN
6,NaN,bf
7,NaN,bf
8,NaN,ff
9,NaN,ff


In [245]:
df3 = df2[df2.isna().any(axis=1)]

In [246]:
df3

,0
2,NaN
3,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [247]:
df4 = df2[~df2.isna().any(axis=1)]

In [228]:
df3.groupby((df3.index.to_series().diff() > 1).cumsum()).transform(lambda x: pd.qcut(x, 2, labels=['bf', 'ff']))

ValueError: transform must return a scalar value for each group

In [248]:
df5 = pd.DataFrame()

In [252]:
for group in grouped:
    test = group[1].reset_index()
    test['fill'] = pd.qcut(test.iloc[:,0], 2, labels=['bf', 'ff'])
    df5 = pd.concat([df5, test])

In [253]:
df5

,index,0,fill
0,2,NaN,bf
1,3,NaN,ff
0,6,NaN,bf
1,7,NaN,bf
2,8,NaN,ff
3,9,NaN,ff


In [256]:
df = pd.DataFrame(s)

In [264]:
df5.set_index('index', inplace=True)

In [271]:
pd.merge(df, df5, how='left', left_index=True, right_index=True).drop(columns=['0_y'])

,0_x,fill
0,1.0,NaN
1,3.0,NaN
2,NaN,bf
3,NaN,ff
4,4.0,NaN
5,6.0,NaN
6,NaN,bf
7,NaN,bf
8,NaN,ff
9,NaN,ff


In [172]:
y = np.array(arr[1:] != arr[:-1]) # pairwise unequal (string safe)

In [173]:
y

array([ True,  True, False,  True,  True,  True, False, False, False,
        True])

In [177]:
i = np.append(np.where(y), len(arr) - 1)   # must include last element posi

In [178]:
i

array([ 0,  1,  3,  4,  5,  9, 10])

In [181]:
s[i]

0     1.0
1     3.0
3     NaN
4     4.0
5     6.0
9     NaN
10    2.0
dtype: float64

In [182]:
z = np.diff(np.append(-1, i))       # run lengths

In [183]:
z

array([1, 1, 2, 1, 1, 4, 1])

In [184]:
p = np.cumsum(np.append(0, z))[:-1] 

In [185]:
p

array([ 0,  1,  2,  4,  5,  6, 10])

In [186]:
s[p]

0     1.0
1     3.0
2     NaN
4     4.0
5     6.0
6     NaN
10    2.0
dtype: float64

In [142]:
s.isna().cumsum()

0     0
1     0
2     1
3     2
4     2
5     2
6     3
7     4
8     5
9     6
10    6
dtype: int64

In [144]:
df = pd.DataFrame(s)

In [146]:
df['hmm'] = df.apply(lambda x: 1 if x.isna() else 0)

ValueError: ('The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().', 'occurred at index 0')

In [139]:
s[s.isna()]

2   NaN
3   NaN
dtype: float64

In [128]:
name_to_df = {}
for i in raw.keys():
    try:
        name_to_df[i] = pd.read_csv('old/'+ raw[i])
    except UnicodeDecodeError:
        name_to_df[i] = pd.read_csv('old/'+ raw[i], encoding='iso-8859-1')

In [129]:
pre_cleaned_target_df = name_to_df['pre_cleaned_target_df']
raw_contract_details = name_to_df['raw_contract_details']

In [101]:
def get_cols(df):
    new_cols = dict(zip(list(df), [re.sub(' ', '_',x.lower()) for x in list(df)]))
    df = df.copy().rename(columns=new_cols)
    relevant = list(set(list(df)) & set(out_cols))
    return df[relevant]

In [106]:
pd.concat([get_cols(pre_cleaned_target_df), get_cols(raw_contract_details)], sort=True).drop_duplicates(keep='first')

,account_birth_date,churn_in_stage,churn_in_stage_paying,churned_date,client_id,date,pai_account_id,pai_is_deleted,poc_end_date,profile,reached_poc_stage,recipe_execution_date,stage,username
0,2018-02-01,0.0,0.0,NaN,3.0,2018-08-07,001D0000029fcDh,0.0,2018-02-01,enterprise,0.0,2018-08-19,paying,yuuta.narikawa@toranoana.co.jp
1,2018-02-27,0.0,0.0,NaN,3.0,2018-08-07,001D000002BC2R8,0.0,2018-02-27,enterprise,0.0,2018-08-19,paying,yuto.kikuchi@aktsk.jp
2,2017-09-30,0.0,0.0,NaN,3.0,2018-08-07,001D000001vOPhg,0.0,2017-09-30,enterprise,0.0,2018-08-19,paying,yutaka.kaneko@sonybank.co.jp
3,2017-09-27,0.0,0.0,NaN,3.0,2018-08-07,001D000001Um4IT,0.0,2017-09-27,enterprise,0.0,2018-08-19,paying,yasushi.watanuki@g.softbank.co.jp
4,2016-12-28,1.0,1.0,2017-12-13,3.0,2018-08-07,001D000001m5Rsz,0.0,2016-12-28,enterprise,0.0,2018-08-19,paying,workiva@clicktale.com
5,2017-03-31,1.0,1.0,2018-06-15,3.0,2018-08-07,001D000001JDMYn,0.0,2017-03-31,enterprise,0.0,2018-08-19,paying,VfCorp@Parent.com
6,2018-03-31,0.0,0.0,NaN,3.0,2018-08-07,001D000001rZ6o7,0.0,2018-03-31,enterprise,0.0,2018-08-19,paying,ui-design.u@sbr-inc.co.jp
7,2017-01-16,1.0,1.0,2018-01-30,3.0,2018-08-07,001D000001MckBJ,0.0,2017-01-16,enterprise,0.0,2018-08-19,paying,tyler@poppin.com
8,NaN,0.0,0.0,NaN,3.0,2018-08-07,001D000002BmZbv,0.0,NaN,enterprise,0.0,2018-08-19,paying,toshihiro.shimizu@rakuten.com
9,2018-06-04,0.0,0.0,NaN,3.0,2018-08-07,001D0000014sASK,0.0,2018-06-04,enterprise,0.0,2018-08-19,paying,t-mobile@clicktale.com


In [97]:
df = pre_cleaned_target_df

In [98]:
new_cols = dict(zip(list(df), [re.sub(' ', '_',x.lower()) for x in list(df)]))

In [100]:
df.copy().rename(columns=new_cols)

,date,client_id,pai_account_id,recipe_execution_date,pai_is_deleted,username,reached_poc_stage,churn_in_stage_paying,stage,churn_in_stage,account_birth_date,poc_end_date,churned_date,profile
0,2018-08-07,3,001D0000029fcDh,2018-08-19,0,yuuta.narikawa@toranoana.co.jp,0,0,paying,0.0,2018-02-01,2018-02-01,NaN,enterprise
1,2018-08-07,3,001D000002BC2R8,2018-08-19,0,yuto.kikuchi@aktsk.jp,0,0,paying,0.0,2018-02-27,2018-02-27,NaN,enterprise
2,2018-08-07,3,001D000001vOPhg,2018-08-19,0,yutaka.kaneko@sonybank.co.jp,0,0,paying,0.0,2017-09-30,2017-09-30,NaN,enterprise
3,2018-08-07,3,001D000001Um4IT,2018-08-19,0,yasushi.watanuki@g.softbank.co.jp,0,0,paying,0.0,2017-09-27,2017-09-27,NaN,enterprise
4,2018-08-07,3,001D000001m5Rsz,2018-08-19,0,workiva@clicktale.com,0,1,paying,1.0,2016-12-28,2016-12-28,2017-12-13,enterprise
5,2018-08-07,3,001D000001JDMYn,2018-08-19,0,VfCorp@Parent.com,0,1,paying,1.0,2017-03-31,2017-03-31,2018-06-15,enterprise
6,2018-08-07,3,001D000001rZ6o7,2018-08-19,0,ui-design.u@sbr-inc.co.jp,0,0,paying,0.0,2018-03-31,2018-03-31,NaN,enterprise
7,2018-08-07,3,001D000001MckBJ,2018-08-19,0,tyler@poppin.com,0,1,paying,1.0,2017-01-16,2017-01-16,2018-01-30,enterprise
8,2018-08-07,3,001D000002BmZbv,2018-08-19,0,toshihiro.shimizu@rakuten.com,0,0,paying,0.0,NaN,NaN,NaN,enterprise
9,2018-08-07,3,001D0000014sASK,2018-08-19,0,t-mobile@clicktale.com,0,0,paying,0.0,2018-06-04,2018-06-04,NaN,enterprise


In [95]:
get_cols(pre_cleaned_target_df)

['date',
 'pai_account_id',
 'recipe_execution_date',
 'reached_poc_stage',
 'account_birth_date',
 'username',
 'profile',
 'churn_in_stage',
 'client_id',
 'poc_end_date',
 'churned_date',
 'stage',
 'pai_is_deleted',
 'churn_in_stage_paying']

In [96]:
pre_cleaned_target_df[['date',
 'pai_account_id',
 'recipe_execution_date',
 'reached_poc_stage',
 'account_birth_date',
 'username',
 'profile',
 'churn_in_stage',
 'client_id',
 'poc_end_date',
 'churned_date',
 'stage',
 'pai_is_deleted',
 'churn_in_stage_paying']]

KeyError: "['churned_date'] not in index"

In [81]:
out_cols = ['account_birth_date',
 'Churned Date',
 'churn_in_stage',
 'churn_in_stage_paying',
 'churn_in_stage_poc',
 'client_id',
 'date',
 'is_deleted',
 'pai_account_id',
 'pai_is_deleted',
 'poc_end_date',
 'profile',
 'recipe_execution_date',
 'stage',
 'username',
 'reached_poc_stage']
out_cols = [re.sub(' ', '_',x.lower()) for x in out_cols]

In [85]:
set(get_cols(raw_contract_details)) & set(out_cols)

{'churned_date', 'username'}

In [84]:
set(get_cols(pre_cleaned_target_df)) & set(out_cols)

{'account_birth_date',
 'churn_in_stage',
 'churn_in_stage_paying',
 'churned_date',
 'client_id',
 'date',
 'pai_account_id',
 'pai_is_deleted',
 'poc_end_date',
 'profile',
 'reached_poc_stage',
 'recipe_execution_date',
 'stage',
 'username'}

In [86]:
len(out_cols)

16

In [70]:
[list(raw_contract_details), list(pre_cleaned_target_df)]

[['Username',
  'Account Name',
  'Contract Start Date',
  'Contract End Date',
  'Payment Terms',
  'ACV',
  'Booking Amount ($)',
  'Account Stage',
  'Invoice Due Date',
  'Invoice Date',
  'Churned Date',
  'History (days)',
  'New Cases Last 6 Months',
  'Contract Months',
  'Contract Type',
  'Contract Sub Type',
  'Financial Month',
  'Contract Owner',
  'Estimated Monthly Traffic',
  'Monthly PV (millions)',
  'Analyses sold',
  'Report sold',
  'Deployment Hours Sold',
  'Total Consulting Hours'],
 ['date',
  'client_id',
  'pai_account_id',
  'recipe_execution_date',
  'pai_is_deleted',
  'username',
  'reached_poc_stage',
  'churn_in_stage_paying',
  'stage',
  'churn_in_stage',
  'account_birth_date',
  'poc_end_date',
  'Churned Date',
  'profile']]

In [63]:
name_to_df['raw_contract_details'].head()

,Username,Account Name,Contract Start Date,Contract End Date,Payment Terms,ACV,Booking Amount ($),Account Stage,Invoice Due Date,Invoice Date,...,Contract Type,Contract Sub Type,Financial Month,Contract Owner,Estimated Monthly Traffic,Monthly PV (millions),Analyses sold,Report sold,Deployment Hours Sold,Total Consulting Hours
0,clicktale2@consumersenergy.com,Consumers Energy,01/11/2017,31/12/2018,30,102857.14,120000,Existing Customer,NaN,NaN,...,Enterprise,New,October,Bret Chappel,NaN,5.0,0.0,NaN,230.0,135.0
1,Clicktale@sky.it,Sky Italia,25/06/2018,24/06/2019,Net +45,138161.60,138162,Existing Customer,NaN,NaN,...,Enterprise,Renewal,NaN,Harold Solomons,90.0,10.0,NaN,NaN,300.0,100.0
2,clicktale@hm.com,H & M Hennes & Mauritz AB,11/10/2017,21/06/2018,NaN,0.00,0,Existing Customer,NaN,NaN,...,Enterprise,Addition,October,Andrew Linfoot,NaN,0.0,0.0,NaN,0.0,50.0
3,Clicktale@hsn.com,HSN Inc.,31/12/2017,30/12/2018,45 days,195000.00,195000,Existing Customer,NaN,NaN,...,Enterprise,Renewal,December,Assaf Rimon,NaN,20.0,0.0,NaN,200.0,150.0
4,clicktale@hellobank.be,Hello Bank Belgium (BNP),18/05/2017,18/05/2019,30,335700.00,671400,Existing Customer,NaN,NaN,...,Enterprise,New,June,Amir Erlich,NaN,1.3,0.0,NaN,500.0,750.0


In [15]:
key_cols = list(pre_contact)

In [23]:
cols = {i:list(name_to_df[i]) for i in name_to_df.keys()}

In [34]:
key_cols

['account_name', 'client_id', 'pai_account_id', 'username']

In [37]:
cols.keys()

dict_keys(['raw_cases', 'pre_account', 'raw_clicktale_accounts', 'raw_accounts_details'])

In [38]:
cols['raw_accounts_details']

['Username',
 'Account ID',
 'Account Owner',
 'Account Name',
 'Account Stage',
 'Type',
 'Employees',
 'Source Channel',
 'Industry',
 'sub industry',
 'Country',
 'Sales Territory',
 'Region',
 'Number of Opps',
 'CEC',
 'Onboarding Manager',
 'CSM',
 'Last Month Recordings']

In [36]:
for i in cols.keys():
    print(i)
    overlap = [x for x in cols[i] if x in key_cols]
    print(['No overlap' if not overlap else overlap])        

raw_cases
['No overlap']
pre_account
[['client_id', 'pai_account_id', 'username', 'account_name']]
raw_clicktale_accounts
['No overlap']
raw_accounts_details
['No overlap']


In [44]:
overlap = [['ACCOUNT_NAME', 'CLIENT_ID', 'PAI_ACCOUNT_ID', 'RECIPE_EXECUTION_DATE', 'USERNAME']]

In [52]:
print(list(name_to_df['pre_account']))

['client_id', 'recipe_execution_date', 'pai_account_id', 'username', 'country', 'account_name', 'total_consulting_hours', 'cx_hours', 'services_sold', '%_pvs_used', 'deployment_hours_sold', 'deployment_delivered', 'monthly_pv_in_millions', 'number_of_reports_sold', 'number_of_analyses_sold', 'tech_lead', 'cec', 'csm', 'last_month_recordings', '%_pvs_Used_to_totango']


In [58]:
name_to_df['pre_account'].columns.tolist() == list(name_to_df['pre_account'])

True

In [49]:
name_to_df['pre_account'][overlap].head()

KeyError: "[('ACCOUNT_NAME', 'CLIENT_ID', 'PAI_ACCOUNT_ID', 'RECIPE_EXECUTION_DATE', 'USERNAME')] not in index"

In [51]:
type(overlap[0])

list

In [48]:
for i in overlap[0]:
    print(i)

ACCOUNT_NAME
CLIENT_ID
PAI_ACCOUNT_ID
RECIPE_EXECUTION_DATE
USERNAME
